# Pluto V2
Trying cleaner data. Will split by each flow. 

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import csv

In [44]:
pcap_dfs = {}
with open(r'data\pluto\new-bidirectional\files.txt', newline='') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    next(reader, None)
    for file_base, label in reader:
        pcap_dfs[file_base] = pd.read_csv(f"data\\pluto\\new-bidirectional\\{file_base}.csv")

In [45]:
pcap_dfs['AmazonPrime11']

,five-tuple,timestamp,packet_size,label
0,NaN,1.703400e+09,60,0
1,"192.168.1.7,239.255.255.250,51772,1900,UDP",1.703400e+09,217,0
2,NaN,1.703400e+09,60,0
3,"192.168.1.43,18.154.132.3,51465,443,TCP",1.703400e+09,880,0
4,"18.154.132.3,192.168.1.43,443,51465,TCP",1.703400e+09,2934,0
...,...,...,...,...
11735,"192.168.1.43,18.154.132.3,51466,443,TCP",1.703400e+09,54,0
11736,"192.168.1.43,18.154.132.3,51466,443,TCP",1.703400e+09,54,0
11737,"18.154.132.3,192.168.1.43,443,51466,TCP",1.703400e+09,2934,0
11738,"18.154.132.3,192.168.1.43,443,51466,TCP",1.703400e+09,2934,0


In [18]:
def seperate_flows(pcap_df):
    return dict(tuple(pcap_df.groupby('five-tuple')))

In [19]:
ap11_flows = seperate_flows(pcap_dfs['AmazonPrime11'])

In [55]:
flow_dfs = {}
for pcap_key in pcap_dfs.keys():
    flows = seperate_flows(pcap_dfs[pcap_key])
    max_length_df_key = max(flows, key=lambda key: len(flows[key]))
    flow_dfs[pcap_key] = flows[max_length_df_key]

In [57]:
flow_dfs

{'AmazonPrime11':                                     five-tuple     timestamp  packet_size  \
 4      18.154.132.3,192.168.1.43,443,51465,TCP  1.703400e+09         2934   
 5      18.154.132.3,192.168.1.43,443,51465,TCP  1.703400e+09         1494   
 7      18.154.132.3,192.168.1.43,443,51465,TCP  1.703400e+09         2934   
 8      18.154.132.3,192.168.1.43,443,51465,TCP  1.703400e+09         1494   
 11     18.154.132.3,192.168.1.43,443,51465,TCP  1.703400e+09         2934   
 ...                                        ...           ...          ...   
 10150  18.154.132.3,192.168.1.43,443,51465,TCP  1.703400e+09          620   
 10151  18.154.132.3,192.168.1.43,443,51465,TCP  1.703400e+09        15894   
 10152  18.154.132.3,192.168.1.43,443,51465,TCP  1.703400e+09          620   
 10157  18.154.132.3,192.168.1.43,443,51465,TCP  1.703400e+09         1494   
 10158  18.154.132.3,192.168.1.43,443,51465,TCP  1.703400e+09           60   
 
        label  
 4          0  
 5          0

In [53]:
# Remove flows with few packets

# FLOW_FILTER_CONST = 10

# filtered_flows = {}

# for flow_key in ap11_flows.keys():
#     flow = ap11_flows[flow_key]
#     total_size = len(ap11_flows[flow_key])
#     total_time = flow['timestamp'].max() - flow['timestamp'].min()
#     size_cutoff = (total_time + 1) * FLOW_FILTER_CONST
#     if total_size > size_cutoff:
#         print(total_size, total_time)
#         filtered_flows[flow_key] = flow

In [50]:
max_length_df_key = max(ap11_flows, key=lambda key: len(ap11_flows[key]))